In [40]:
import pandas as pd

Weather = pd.read_csv("/Weather.csv", index_col="DATE")

FileNotFoundError: ignored

In [32]:
Weather

,station,name,prcp,snow,tavg,tmax,tmin
DATE,,,,,,,
2019-01-01,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,39.0,41.0,37.0
2019-01-02,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,40.0,42.0,38.0
2019-01-03,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,44.0,48.0,40.0
2019-01-04,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,47.0,50.0,44.0
2019-01-05,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,45.0,47.0,43.0
...,...,...,...,...,...,...,...
2022-01-27,CA001105669,"N VANCOUVER WHARVES, BC CA",0.00,0.0,NaN,NaN,NaN
2022-01-28,CA001105669,"N VANCOUVER WHARVES, BC CA",0.00,0.0,NaN,NaN,NaN
2022-01-29,CA001105669,"N VANCOUVER WHARVES, BC CA",0.45,0.0,NaN,NaN,NaN


In [33]:
null_pct = Weather.apply(pd.isnull).sum()/Weather.shape[0]
null_pct

station    0.000000
name       0.000000
prcp       0.086830
snow       0.448398
tavg       0.459367
tmax       0.428481
tmin       0.429951
dtype: float64

In [34]:
valid_columns = Weather.columns[null_pct < .5]

In [35]:
valid_columns

Index(['station', 'name', 'prcp', 'snow', 'tavg', 'tmax', 'tmin'], dtype='object')

In [36]:
Weather = Weather[valid_columns].copy()

In [37]:
Weather.columns = Weather.columns.str.lower()

In [38]:
Weather

,station,name,prcp,snow,tavg,tmax,tmin
DATE,,,,,,,
2019-01-01,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,39.0,41.0,37.0
2019-01-02,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,40.0,42.0,38.0
2019-01-03,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,44.0,48.0,40.0
2019-01-04,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,47.0,50.0,44.0
2019-01-05,CA001107010,"SANDHEADS CS, BC CA",NaN,NaN,45.0,47.0,43.0
...,...,...,...,...,...,...,...
2022-01-27,CA001105669,"N VANCOUVER WHARVES, BC CA",0.00,0.0,NaN,NaN,NaN
2022-01-28,CA001105669,"N VANCOUVER WHARVES, BC CA",0.00,0.0,NaN,NaN,NaN
2022-01-29,CA001105669,"N VANCOUVER WHARVES, BC CA",0.45,0.0,NaN,NaN,NaN


In [39]:
weather = weather.ffill()

NameError: ignored

In [ ]:
weather.apply(pd.isnull).sum()

In [ ]:
weather.apply(lambda x: (x == 9999).sum())

In [ ]:
weather.dtypes

In [ ]:
weather.index

In [ ]:
weather.index = pd.to_datetime(weather.index)

In [ ]:
weather.index.year.value_counts().sort_index()

In [ ]:
weather["snwd"].plot()

In [ ]:
weather["target"] = weather.shift(-1)["tmax"]

In [ ]:
weather

In [ ]:
weather = Weather.ffill()

In [ ]:
weather

In [ ]:
from sklearn.linear_model import Ridge

rr = Ridge(alpha=.5)

In [ ]:
predictors = Weather.columns[~Weather.columns.isin(["target","name", "station"])]

In [ ]:
def backtest(weather, model, predictors, start=3650, step=90):
    all_predictions = []
    
    for i in range(start, weather.shape[0], step):
        train = weather.iloc[:i,:]
        test = weather.iloc[i:(i+step),:]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        combined["diff"] = (combined["prediction"] - combined["actual"]).abs()
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [ ]:
predictions = backtest (weather, rr, predictors)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mean_absolute_error(predictions["actual"], predictions["prediction"])

In [ ]:
predictions.sort_values("diff", ascending=False)

In [ ]:
pd.Series(rr.coef_, index=predictors)

In [ ]:
def pct_diff(old, new):
    return (new - old) / old

def compute_rolling(weather, horizon, col):
    label = f"rolling_{horizon}_{col}"
    weather[label] = weather[col].rolling(horizon).mean()
    weather[f"{label}_pct"] = pct_diff(weather[label], weather[col])
    return weather
    
rolling_horizons = [3, 14]
for horizon in rolling_horizons:
    for col in ["tmax", "tmin", "prcp"]:
        weather = compute_rolling(weather, horizon, col)

In [ ]:
def expand_mean(df):
    return df.expanding(1).mean()

for col in ["tmax", "tmin", "prcp"]:
    weather[f"month_avg_{col}"] = weather[col].groupby(weather.index.month, group_keys=False).apply(expand_mean)
    weather[f"day_avg_{col}"] = weather[col].groupby(weather.index.day_of_year, group_keys=False).apply(expand_mean)

In [ ]:
weather = weather.iloc[14:,:]
weather = weather.fillna(0)

In [ ]:
predictors = weather.columns[~weather.columns.isin(["target", "name", "station"])]

In [ ]:
predictions = backtest(weather, rr, predictors)
mean_absolute_error(predictions["actual"], predictions["prediction"])

In [ ]:
mean_squared_error(predictions["actual"], predictions["prediction"])

In [ ]:
predictions.sort_values("diff", ascending=False)

In [ ]:
weather.loc["2019-01-01": "2022-12-31"]

In [ ]:
(predictions["diff"].round().value_counts().sort_index() / predictions.shape[0]).plot()

In [ ]:
predictions